# Previsão de notas dos alunos
Muitos alunos de pós-graduação têm dificuldade em obter boas notas porque não recebem muito apoio nos cursos superiores em comparação com o apoio que os alunos recebem nas escolas. Alguns alunos precisam de muita atenção dos instrutores para que obtenham boas notas, sem isso, o estado emocional do aluno pode ser prejudicial para a sua carreira a longo prazo.

**O objetivo desse projeto é, através do aprendizado de máquina, prever as notas dos alunos para que os instrutores possam ajudar os alunos a se prepararem para tópicos em que as notas dos alunos foram previstas baixas.**  



https://www.kaggle.com/code/ramontanoeiro/student-performance/notebook

# #3 - Seleção de atributos
Algoritmos que utilizarão a base de dados com numéricos normalizados
- from sklearn.svm import SVR
- from sklearn.neural_network import MLPRegressor
- from sklearn.neighbors import KNeighborsRegressor

Algoritmos que utilizarão a base de dados com numéricos **não** normalizados
- from sklearn.tree import DecisionTreeRegressor
- from sklearn.ensemble import RandomForestRegressor
- from gboost import XGBRegressor

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from sklearn.feature_selection import RFE
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score

import pandas as pd
import numpy as np
import math
import pickle as pkl
import matplotlib.pyplot as plt
path_datasets = '/content/drive/MyDrive/Machine Learning e Data Science com Python/Projeto - Nota dos Alunos/'

## Carregando datasets tratado
- **MA:** Dataset sem features com multicolinearidade alta
- **MMA:** Dataset sem features com multicolinearidade muito alta
- **NN:** Dataset com features numéricas não normalizadas

In [5]:
with open(path_datasets+'students_data.pkl', 'rb') as f:
  [X_train, X_MA_train, X_MMA_train,
   X_test , X_MA_test , X_MMA_test ,

   X_NN_train, X_MA_NN_train, X_MMA_NN_train,
   X_NN_test , X_MA_NN_test , X_MMA_NN_test ,
   
   y_train, y_test] = pkl.load(f)

print(X_train.shape, X_MA_train.shape, X_MMA_train.shape, X_test.shape, X_MA_test.shape, X_MMA_test.shape)
print(X_NN_train.shape, X_MA_NN_train.shape, X_MMA_NN_train.shape, X_NN_test.shape , X_MA_NN_test.shape , X_MMA_NN_test.shape) 
print(y_train.shape, y_test.shape)

(296, 34) (296, 26) (296, 30) (99, 34) (99, 26) (99, 30)
(296, 34) (296, 26) (296, 30) (99, 34) (99, 26) (99, 30)
(296,) (99,)


## RFE
O algoritmo deve fornecer uma maneira de calcular pontuações importantes, como uma árvore de decisão. O algoritmo usado no RFE não precisa ser o algoritmo que se encaixa nos recursos selecionados; diferentes algoritmos podem ser usados.

Uma vez configurada, a classe deve ser ajustada em um conjunto de dados de treinamento para selecionar os recursos chamando a função fit() . Após o ajuste da classe, a escolha das variáveis ​​de entrada pode ser visualizada através do atributo “ support_ ” que fornece um True ou False para cada variável de entrada.

## Seleção de modelos

### Modelos que utilizarão features numéricas normalizadas
**LIMITAÇÕES:** Os modelos: KNeighborsRegressor, MLPRegressor e SVR (exceto kernel linear) não possuem os atributos `coef_` ou `feature_importances_`. Com isso, não é possível realizar a seleção de atributos com esses algoritmos.

In [6]:
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor

### Modelos que utilizarão features numéricas não normalizadas

In [7]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

In [ ]:
list_models = [
      ('svm-lin', SVR(kernel = 'linear'))
    , ('tree', DecisionTreeRegressor())
    , ('forest', RandomForestRegressor())
    , ('xgbr', XGBRegressor()) ]

In [ ]:
# Montando combinações de modelo & features_to_select
def merge_model_features(df):
  models = {}
  n_features_min = df.shape[1]
  for i in range(1, n_features_min+1):
    for mdl_nm, mdl in list_models:
      rfe = RFE(estimator = mdl, n_features_to_select = i)
      models[f'{mdl_nm}_{i}'] = Pipeline(steps=[('rfe', rfe),('mdl', mdl)])
  
  return models

In [ ]:
models_NN     = merge_model_features(X_NN_train)
models_MA_NN  = merge_model_features(X_MA_NN_train)
models_MMA_NN = merge_model_features(X_MMA_NN_train)
len(models_NN), len(models_MA_NN), len(models_MMA_NN)

(136, 104, 120)

Vamos utilizar a métrica **neg_mean_absolute_error** para avavliar a seleção de features. Como MAE é uma métrica de erro, ou seja, quanto menor melhor, MAE negativo é o contrário: um valor de -2.6 é melhor que um valor de -3.0.  

Vamos utilizar o método de validação cruzada KFold que fornece índices de treinamento e validação para dividir o connjunto de dados em k dobras consecutivas. Cada dobra é utilizada uma vez como validação enquanto as outras K-1 formam o conjunto de treinamento.

In [ ]:
# Calculando combinações de features e modelo
def features_to_select(models, verbose = False):
  seed = 14
  model_scores = []

  for mdl_nm, mdl_pipeline in models.items():
    print(f'Model: {mdl_nm.split("_")[0]}\tFeatures to Select: {mdl_nm.split("_")[1]}') if verbose else None
    kfold = KFold(n_splits = 5, shuffle = True, random_state = seed)
    score = cross_val_score(mdl_pipeline, X_train, y_train, cv = kfold, n_jobs=4, error_score='raise', scoring = 'neg_mean_absolute_error')
    model_scores.append((mdl_nm, score.mean(), score.std()))
  
  return model_scores

In [ ]:
model_scores_NN     = features_to_select(models_NN, verbose = True)
model_scores_MA_NN  = features_to_select(models_MA_NN)
model_scores_MMA_NN = features_to_select(models_MMA_NN)

Model: svm-lin	Features to Select: 1
Model: tree	Features to Select: 1
Model: forest	Features to Select: 1
Model: xgbr	Features to Select: 1
Model: svm-lin	Features to Select: 2
Model: tree	Features to Select: 2
Model: forest	Features to Select: 2
Model: xgbr	Features to Select: 2
Model: svm-lin	Features to Select: 3
Model: tree	Features to Select: 3
Model: forest	Features to Select: 3
Model: xgbr	Features to Select: 3
Model: svm-lin	Features to Select: 4
Model: tree	Features to Select: 4
Model: forest	Features to Select: 4
Model: xgbr	Features to Select: 4
Model: svm-lin	Features to Select: 5
Model: tree	Features to Select: 5
Model: forest	Features to Select: 5
Model: xgbr	Features to Select: 5
Model: svm-lin	Features to Select: 6
Model: tree	Features to Select: 6
Model: forest	Features to Select: 6
Model: xgbr	Features to Select: 6
Model: svm-lin	Features to Select: 7
Model: tree	Features to Select: 7
Model: forest	Features to Select: 7
Model: xgbr	Features to Select: 7
Model: svm-li

## Obtendo resultados

In [ ]:
# Transformando resultado em DataFrame
def df_model_scores(model_scores):
  data_df = []
  for m_f, f1, std in model_scores:
    m, f = m_f.split('_')
    data_df.append((m, f, f1, std))
  rfe_results = pd.DataFrame(data_df, columns = ['model', 'num_features', 'neg_MAE', 'std'])
  rfe_results.sort_values(by=['neg_MAE'], ascending = False, inplace = True)
  return rfe_results

In [ ]:
df_model_scores_NN     = df_model_scores(model_scores_NN)
df_model_scores_MA_NN  = df_model_scores(model_scores_MA_NN)
df_model_scores_MMA_NN = df_model_scores(model_scores_MMA_NN)

## Analisando resultados

### X_train_NN
Base de dados sem normalização

In [ ]:
# Melhores resultados
df_model_scores_NN.head(10)

,model,num_features,neg_MAE,std
30,forest,8,-0.902793,0.134977
7,xgbr,2,-0.911074,0.177540
102,forest,26,-0.911501,0.138588
18,forest,5,-0.916029,0.192167
34,forest,9,-0.918086,0.150216
94,forest,24,-0.921754,0.149373
14,forest,4,-0.922959,0.166936
42,forest,11,-0.923286,0.124039
114,forest,29,-0.924859,0.151390
122,forest,31,-0.924893,0.158653


In [ ]:
# Melhor resultado de cada algoritmo
df_best_NN = pd.concat([df_model_scores_NN[ df_model_scores_NN['model'] == 'forest'].head(1),
           df_model_scores_NN[ df_model_scores_NN['model'] == 'xgbr'].head(1),
           df_model_scores_NN[ df_model_scores_NN['model'] == 'svm-lin'].head(1),
           df_model_scores_NN[ df_model_scores_NN['model'] == 'svm-poly'].head(1),
           df_model_scores_NN[ df_model_scores_NN['model'] == 'tree'].head(1)], ignore_index=True).sort_values(by=['neg_MAE'], ascending = False)
df_best_NN['database'] = 'NN'

### X_train_MA_NN
Base de dados sem atributos com multicolinearidade alta e sem normalização

In [ ]:
# Melhores resultados
df_model_scores_MA_NN.head(10)

,model,num_features,neg_MAE,std
7,xgbr,2,-0.911074,0.177540
38,forest,10,-0.911856,0.131637
26,forest,7,-0.913580,0.138664
30,forest,8,-0.914685,0.145361
42,forest,11,-0.925620,0.128903
98,forest,25,-0.926500,0.156947
22,forest,6,-0.926741,0.160408
46,forest,12,-0.928388,0.116441
70,forest,18,-0.928854,0.124433
82,forest,21,-0.928973,0.131984


In [ ]:
# Melhor resultado de cada algoritmo
df_best_MA_NN = pd.concat([df_model_scores_MA_NN[ df_model_scores_MA_NN['model'] == 'forest'].head(1),
           df_model_scores_MA_NN[ df_model_scores_MA_NN['model'] == 'xgbr'].head(1),
           df_model_scores_MA_NN[ df_model_scores_MA_NN['model'] == 'svm-lin'].head(1),
           df_model_scores_MA_NN[ df_model_scores_MA_NN['model'] == 'tree'].head(1)], ignore_index=True).sort_values(by=['neg_MAE'], ascending = False)
df_best_MA_NN['database'] = 'MA_NN'

### X_train_MMA_NN
Base de dados sem atributos com multicolinearidade muito alta e sem normalização

In [ ]:
# Melhores resultados
df_model_scores_MMA_NN.head(10)

,model,num_features,neg_MAE,std
18,forest,5,-0.907412,0.128192
34,forest,9,-0.910046,0.131147
7,xgbr,2,-0.911074,0.177540
98,forest,25,-0.912979,0.137771
102,forest,26,-0.917948,0.154364
106,forest,27,-0.922407,0.130725
22,forest,6,-0.925138,0.131405
6,forest,2,-0.926077,0.175564
62,forest,16,-0.927231,0.160735
50,forest,13,-0.928899,0.127282


In [ ]:
# Melhor resultado de cada algoritmo
df_best_MMA_NN = pd.concat([df_model_scores_MMA_NN[ df_model_scores_MMA_NN['model'] == 'forest'].head(1),
           df_model_scores_MMA_NN[ df_model_scores_MMA_NN['model'] == 'xgbr'].head(1),
           df_model_scores_MMA_NN[ df_model_scores_MMA_NN['model'] == 'svm-lin'].head(1),
           df_model_scores_MMA_NN[ df_model_scores_MMA_NN['model'] == 'tree'].head(1)], ignore_index=True).sort_values(by=['neg_MAE'], ascending = False)
df_best_MMA_NN['database'] = 'MMA_NN'

Com base do resultado da métrica Mean Absolute Error, vamos selecionar o melhor resultado de cada modelo.  

| Modelo        | Num Features | Base de dados                                          | Motivo                                                                                                                                |
|---------------|--------------|--------------------------------------------------------|---------------------------------------------------------------------------------------------------------------------------------------|
| Random Forest | 5            | Atributos com multicolinearidade muito alta removidos  | Apesar de ser o 2º melhor desempenho entre os testes do RF, ele utiliza menos recursos e possui desvio padrão menor que o 1º colocado |
| XGBR          | 2            | Base de dados sem remoção de atributos multicolineares | 3 base de dados obteve mesmo melhor resultado                                                                                         |
| Decision Tree | 2            | Base de dados sem remoção de atributos multicolineares | 3 base de dados obteve mesmo melhor resultado                                                                                         |
| SVM Linear    | 1            | Base de dados sem remoção de atributos multicolineares | 3 base de dados obteve mesmo melhor resultado                                                                                         |

In [ ]:
df_best = pd.concat([df_best_NN, df_best_MA_NN, df_best_MMA_NN]).sort_values(by=['neg_MAE'], ascending = False, ignore_index = True)
df_best

,model,num_features,neg_MAE,std,database
0,forest,8,-0.902793,0.134977,NN
1,forest,5,-0.907412,0.128192,MMA_NN
2,xgbr,2,-0.911074,0.177540,NN
3,xgbr,2,-0.911074,0.177540,MA_NN
4,xgbr,2,-0.911074,0.177540,MMA_NN
5,forest,10,-0.911856,0.131637,MA_NN
6,tree,2,-0.931862,0.178275,NN
7,tree,2,-0.935252,0.175300,MA_NN
8,tree,2,-0.935252,0.175300,MMA_NN
9,svm-lin,1,-0.960181,0.160125,NN


Realizando o treinamento para capturar as features selecionadas por cada modelo


In [8]:
rfe_forest_MMA_NN = RFE(estimator=RandomForestRegressor(), n_features_to_select=5)
rfe_xgbr_NN    = RFE(XGBRegressor(), n_features_to_select = 2)
rfe_tree_NN    = RFE(DecisionTreeRegressor(), n_features_to_select = 2) 
rfe_svm_lin_NN = RFE(SVR(kernel = 'linear'), n_features_to_select = 1)

In [ ]:
rfe_forest_MMA_NN.fit(X_MMA_NN_train, y_train.ravel());
rfe_xgbr_NN.fit(X_NN_train, y_train.ravel());
rfe_tree_NN.fit(X_NN_train, y_train.ravel());
rfe_svm_lin_NN.fit(X_NN_train, y_train.ravel());

**Support:** A máscara dos recursos selecionados.  
**Ranking:** Ranking dos recursos selecionados.

In [10]:
def get_sup_rnk(mdl_nm, df, rfe):
  sup_rank = []
  for i in range(df.shape[1]):
    sup_rank.append((mdl_nm, df.columns[i], rfe.support_[i], rfe.ranking_[i]))
  new_df = pd.DataFrame(sup_rank, columns = ['model', 'feature', 'support', 'ranking'])
  return new_df, list(new_df[new_df['support'] == True]['feature'].values)

In [11]:
forest_sup_rnk, rfe_forest_columns   = get_sup_rnk('forest', X_MMA_NN_train, rfe_forest_MMA_NN)
xgbr_sup_rnk, rfe_xgbr_columns       = get_sup_rnk('xgbr', X_NN_train, rfe_xgbr_NN)
tree_sup_rnk, rfe_tree_columns       = get_sup_rnk('tree', X_NN_train, rfe_tree_NN)
svm_lin_sup_rnk, rfe_svm_lin_columns = get_sup_rnk('svm-lin', X_NN_train, rfe_svm_lin_NN)

In [12]:
# Features selecionadas no modelo Random Forest
forest_sup_rnk[ forest_sup_rnk['support'] == True ]

,model,feature,support,ranking
4,forest,failures,True,1
5,forest,freetime,True,1
6,forest,goout,True,1
9,forest,health,True,1
10,forest,absences,True,1


In [13]:
# Features selecionadas no modelo XGBR
xgbr_sup_rnk[ xgbr_sup_rnk['support'] == True ]

,model,feature,support,ranking
12,xgbr,absences,True,1
14,xgbr,G2,True,1


In [14]:
# Features selecionadas no modelo Decision Tree
tree_sup_rnk[ tree_sup_rnk['support'] == True ]

,model,feature,support,ranking
12,tree,absences,True,1
14,tree,G2,True,1


In [15]:
# Features selecionadas no modelo SVM Linear
svm_lin_sup_rnk[ svm_lin_sup_rnk['support'] == True ]

,model,feature,support,ranking
14,svm-lin,G2,True,1


## Salvando

In [16]:
X_MMA_NN_forest_train, X_MMA_NN_forest_test = X_MMA_NN_train[rfe_forest_columns], X_MMA_NN_test[rfe_forest_columns]
X_NN_xgbr_train, X_NN_xgbr_test   = X_NN_train[rfe_xgbr_columns], X_NN_test[rfe_xgbr_columns]
X_NN_tree_train, X_NN_tree_test   = X_NN_train[rfe_tree_columns], X_NN_test[rfe_tree_columns]
X_NN_svm_lin_train, X_NN_svm_lin_test = X_NN_train[rfe_svm_lin_columns], X_NN_test[rfe_svm_lin_columns]

In [17]:
with open(path_datasets+'students_data_2.pkl', 'wb') as f:
  pkl.dump([X_train, X_MA_train, X_MMA_train,
            X_test , X_MA_test , X_MMA_test ,

            X_MMA_NN_forest_train, X_MMA_NN_forest_test,
            X_NN_xgbr_train, X_NN_xgbr_test,
            X_NN_tree_train, X_NN_tree_test,
            X_NN_svm_lin_train, X_NN_svm_lin_test,

            y_train, y_test], f)
print('OK')

OK
